In [49]:
%pip install --upgrade --quiet  azure-search-documents
%pip install --upgrade --quiet  azure-identity


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [50]:
import os
from langchain_community.vectorstores.azuresearch import AzureSearch
from langchain_openai import AzureOpenAIEmbeddings, OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter

In [51]:
azure_endpoint: str = "https://demoopenairesource.openai.azure.com/"
azure_openai_api_key: str = "55FnF6s4J4OywbWkhBg75P6iFekRErJK3IYW40zlwD5EfUnFoH5MJQQJ99BBACHYHv6XJ3w3AAABACOGEtm9"
azure_openai_api_version: str = "2023-05-15"
azure_deployment: str = "text-embedding-3-small"

In [52]:
loader = PyPDFLoader("2WPolicy.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

In [53]:
embeddings: AzureOpenAIEmbeddings = AzureOpenAIEmbeddings(
    azure_deployment=azure_deployment,
    openai_api_version=azure_openai_api_version,
    azure_endpoint=azure_endpoint,
    api_key=azure_openai_api_key,
)

vector_data = [
    {"id": str(i), "content": chunk.page_content, "vector": embeddings.embed_query(chunk.page_content)}
    for i, chunk in enumerate(docs)
]


In [54]:
len(vector_data), len(vector_data[0]['vector'])

(13, 1536)

In [55]:
vector_store_address: str = "https://testaisearchmctltfs030225.search.windows.net"
vector_store_password: str = "X5ayCxEXO3KNEKDgUp6hn466Gd7iDsukNQujhRfXV2AzSeDNKv8a"

In [56]:
index_name: str = "two-weheeler-policy-index"
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

In [57]:
index_name

'two-weheeler-policy-index'

In [58]:
# # Specify additional properties for the Azure client such as the following https://github.com/Azure/azure-sdk-for-python/blob/main/sdk/core/azure-core/README.md#configurations
# vector_store: AzureSearch = AzureSearch(
#     azure_search_endpoint=vector_store_address,
#     azure_search_key=vector_store_password,
#     index_name=index_name,
#     embedding_function=embeddings.embed_query,
#     # Configure max retries for the Azure client
#     additional_search_client_options={"retry_total": 4},
# )

In [59]:
# vector_store.client.delete_documents(keys=[index_name])

In [60]:
import json

# Convert documents to vector format
vector_data = [
    {
        "id": str(i),  # Unique ID for each document
        "content": doc.page_content,  # Original text for keyword search
        "vector": embeddings.embed_query(doc.page_content),  # Embedding vector
    }
    for i, doc in enumerate(docs)  # docs is your list of Document objects
]

# Now upload vector_data instead of docs
vector_store.client.upload_documents(documents=vector_data)

In [61]:
# vector_store.add_documents(documents=vector_data)

In [62]:
# vector_data

### CSV Data

In [64]:
from datetime import datetime

def convert_to_iso(date_str):
    """Convert 'DD-MMM-YYYY' to 'YYYY-MM-DDT00:00:00Z'"""
    try:
        return datetime.strptime(date_str, "%d-%b-%Y").strftime("%Y-%m-%dT00:00:00Z")
    except ValueError:
        return None  # Handle invalid dates

In [65]:
data = [
    {
        "agreement_number": "AG12345",
        "customer_name": "Rajesh Kumar",
        "contact_number": "9876543210",
        "emi_due": "3500",
        "last_emi_due_date": convert_to_iso("03-Feb-2025"),
        "last_emi_status": "Paid",
        "last_emi_paid_date": convert_to_iso("03-Feb-2025"),
        "next_emi_due_date": convert_to_iso("03-Mar-2025"),
        "outstanding_amount": "25000"
    },
    {
        "agreement_number": "AG67890",
        "customer_name": "Priya Sharma",
        "contact_number": "9823456789",
        "emi_due": "2800",
        "last_emi_due_date": convert_to_iso("03-Feb-2025"),
        "last_emi_status": "Pending",
        "last_emi_paid_date": convert_to_iso("10-Jan-2025"),
        "next_emi_due_date": convert_to_iso("03-Mar-2025"),
        "outstanding_amount": "18000"
    },
    {
        "agreement_number": "AG13579",
        "customer_name": "Aditi",
        "contact_number": "123456",
        "emi_due": "1500",
        "last_emi_due_date": convert_to_iso("03-Feb-2025"),
        "last_emi_status": "Pending",
        "last_emi_paid_date":convert_to_iso("03-Dec-2024"),
        "next_emi_due_date": convert_to_iso("03-Mar-2025"),
        "outstanding_amount": "25000"
    }
]


In [68]:
import json
from langchain_community.vectorstores import AzureSearch

# Define your vector store (Ensure correct credentials)
index_name = "loan-index"
vector_store = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

# Uploading Data (List of Dicts, Not JSON String)
vector_store.client.upload_documents(documents=data)  # ✅ Uploads correctly


In [72]:
from azure.search.documents.indexes import SearchIndexClient
from azure.core.credentials import AzureKeyCredential

# Azure Search Config
search_client = SearchIndexClient(
    endpoint=os.environ["AZURE_SEARCH_ENDPOINT"],
    credential=AzureKeyCredential(os.environ["AZURE_SEARCH_KEY"])
)

index_schema = search_client.get_index(index_name)

# Print Available Fields
print([field.name for field in index_schema.fields])

['agreement_number', 'customer_name', 'contact_number', 'emi_due', 'last_emi_due_date', 'last_emi_status', 'last_emi_paid_date', 'next_emi_due_date', 'outstanding_amount']
